Check bản ghi doanh so

In [1]:
from src.Get_data_DB import DataTransformer
import pandas as pd
# Tạo instance của class
transformer = DataTransformer()

# Lấy dữ liệu lần đầu từ năm 2024 Mysql
mysql_query = """SELECT 
                      tp.id AS Id,
                      o.final_price AS Doanh_so,
                      DATE_ADD(t.payment_at, INTERVAL 7 HOUR) AS Ngay_chuyen_tien,
                      o.name as Ten_khach,
                      o.email as Khach_hang,
                      o.mobile as So_dt,
                      p.code AS Ten_khoa_hoc,
                      o.current_sale_id AS Ma_saler,
                      l.utm_medium AS Ma_marketer,
                      o.source AS Ma_kenh
    FROM transactions t
    JOIN transactions_products tp ON tp.p_transaction_id = t.id
    JOIN products p ON tp.product_id = p.id
    JOIN orders o ON t.order_id = o.id
    JOIN leads l ON o.lead_id = l.id
    join users u on o.current_sale_id=u.id
    WHERE t.t_status = 1
      AND DATE_ADD(t.payment_at, INTERVAL 7 HOUR) >= '2025-10-01' 
      and DATE_ADD(t.payment_at, INTERVAL 7 HOUR) <= '2025-10-03' 
      And u.role=3
#      and u.name= 'Ánh Lê'
    GROUP BY tp.id, Ten_khoa_hoc, Ma_saler, Ma_marketer, Ma_kenh,Khach_hang,Doanh_so, Ten_khach, So_dt
                    """
df1 = transformer.fetch_from_mysql(mysql_query)
df1

ModuleNotFoundError: No module named 'mysql'

Check dữ liệu Marketing

In [1]:
""" 
Mỗi chiều thì check dữ liệu chi phí của ngày hôm qua cho chính xác
Đối với dữ liệu của những chiến dịch chạy chuyển đổi, kiểm tra như sau:
    - Chi phí: Vào folder data_tmp, tìm 2 file spend_C9_ACTIVE.csv & spend_Cole8_ACTIVE.csv. Kiểm tra xem phần chi phí của từng chiến dịch 
               có khớp với chi phí với trên Meta Ads ko ? ( cho phép sai số 1k VND)
    - Đối với chỉ số L1 thì dùng đoạn code dưới đây để check, hỏi a Bình cách đếm. 
"""

from src.Get_data_DB import DataTransformer
# Tạo instance của class
transformer = DataTransformer()

# Lấy dữ liệu lần đầu từ năm 2024 Mysql  DATE_SUB(NOW(), INTERVAL 3 MONTH)
mysql_query = """SELECT DATE(DATE_ADD(l.created_at, INTERVAL 7 HOUR)) AS Thoi_gian,                                               
                        lp.product_id AS Ma_khoa_hoc,
                        COUNT(DISTINCT CASE WHEN s2.sale_order_level_id = 1 THEN l.id END) AS L1,
                        COUNT(DISTINCT CASE WHEN s2.sale_order_level_id = 1 THEN l.id END) - COUNT(DISTINCT CASE WHEN s2.sale_order_level_id = 3 THEN l.id END) AS L1_L1C,
                        COUNT(DISTINCT CASE WHEN s2.sale_order_level_id = 16 THEN l.id END) AS L7,
                        COUNT(DISTINCT CASE WHEN s2.sale_order_level_id = 19 THEN l.id END) AS L8
                FROM leads l 
                JOIN leads_products lp ON lp.lead_id = l.id
                JOIN orders o ON o.lead_id = l.id
                JOIN sale_order_histories s1 ON s1.order_id = o.id AND s1.sale_order_level_id = 1
                JOIN sale_order_histories s2 ON s2.order_id = o.id 
                WHERE l.status = 1 
                AND DATE(DATE_ADD(l.created_at, INTERVAL 7 HOUR)) >=  DATE_SUB(NOW(), INTERVAL 4 MONTH)
                AND l.utm_source='FA'
                GROUP BY Thoi_gian, Ma_khoa_hoc
                    """
df0 = transformer.fetch_from_mysql(mysql_query)
df0

# Nếu bị lệch số lượng ở chỉ số nào thì e dùng câu truy vấn để check cụ thể từng bản ghi
# Tham số s2.sale_order_level_id e thay tương ứng với Level e muốn check, ở đoạn code dưới là đang check bản ghi L1.C
# Vào apollo.sale_order_levels để xem bảng ánh xạ.
"""
SELECT  DATE(DATE_ADD(l.created_at, INTERVAL 7 HOUR)) AS Thoi_gian,                                               
        lp.product_id AS Ma_khoa_hoc,
        o.name as Ten_khach,
        o.email as Email,
        o.mobile as So_dt
FROM leads l 
JOIN leads_products lp ON lp.lead_id = l.id
JOIN orders o ON o.lead_id = l.id
JOIN sale_order_histories s1 ON s1.order_id = o.id AND s1.sale_order_level_id = 1
JOIN sale_order_histories s2 ON s2.order_id = o.id and s2.sale_order_level_id = 3
WHERE l.status = 1 
  AND DATE(DATE_ADD(l.created_at, INTERVAL 7 HOUR)) = '2025-09-25'
  AND l.utm_source='FA' 
"""


ModuleNotFoundError: No module named 'mysql'